In [21]:
import pandas as pd
import plotly.graph_objects as go
from pandas_datareader import DataReader as pdr
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np

In [19]:
ret = pd.read_csv("rets.csv", index_col="date")
ret = 1.5*ret.best - 0.5*ret.worst

ff = pdr("F-F_Research_Data_Factors", "famafrench", start=2005)[0]/100
ff.index = ff.index.astype(str)
mkt = ff["Mkt-RF"] + ff["RF"]
rf = ff["RF"]

df = pd.concat((ret, mkt, rf), axis=1)
df.columns = ["ret", "mkt", "rf"]

# Sharpe ratio

In [22]:
xret = df.ret - df.rf 
print(f"Annualized Sharpe ratio of 150-50 strategy is {np.sqrt(12)*xret.mean()/xret.std():.2%}")

Annualized Sharpe ratio of 150-50 strategy is 81.38%


# Example Accumulation Plot

In [23]:
trace1 = go.Scatter(
    x = df.index.to_list(),
    y = (1+df.ret).cumprod(),
    mode="lines",
    name="150-50 return",
    hovertemplate="%{x}<br>accumulation = $%{y:.2f}<extra></extra>"
)

trace2 = go.Scatter(
    x = df.index.to_list(),
    y = (1+df.mkt).cumprod(),
    mode="lines",
    name="market return",
    hovertemplate="%{x}<br>accumulation = $%{y:.2f}<extra></extra>"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    yaxis_title="",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.show()

# Example Drawdown Plot

In [24]:
df["accum"] = (1+df.ret).cumprod()
accum_max = df.accum.expanding().max()
df["drawdown"] = df.accum/accum_max - 1

trace1 = go.Scatter(
    x = df.index.to_list(),
    y = df.accum,
    mode="lines",
    name="150-50 accumulation",
    hovertemplate="%{x}<br>accumulation = $%{y:.2f}<extra></extra>"
)

trace2 = go.Scatter(
    x = df.index.to_list(),
    y = df.drawdown,
    mode="lines",
    name="150-50 drawdown",
    hovertemplate="%{x}<br>drawdown = %{y:.1%}<extra></extra>"
)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1, secondary_y=False)
fig.add_trace(trace2, secondary_y=True)

plotly_template = pio.templates["plotly_white"]
colors = plotly_template.layout.colorway

fig.update_yaxes(
    title_text="<b>Accumulation</b>", 
    title_font=dict(color=colors[0]), 
    color=colors[0], 
    tickformat=".2f", 
    tickprefix="$", 
    secondary_y=False
)
fig.update_yaxes(
    title_text="<b>Drawdown</b>", 
    title_font=dict(color=colors[1]), 
    color=colors[1], 
    tickformat=".0%", 
    secondary_y=True
)

fig.update_layout(
    template="plotly_white",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.99,
      y = 0.01
    ),
    font_size=16,
    showlegend=False,
)

fig.show()